In [1]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import geopandas as gpd
from sklearn.cluster import DBSCAN
# from haversine import haversine as hs
import requests
%matplotlib inline
# from pydantic_settings import BaseSettings
#from ydata_profiling import ProfileReport

import sys
import os
sys.path.append('../utils/')
#sys.path.append('..\\utils\\')
from functions import update_start_date_vectorized, main_concurrent_sn

In [2]:
short_trips = pd.read_csv("../data/processed/sn_shorttrips.csv", parse_dates=["ReservationTime", "StartTime", "EndTime"])
short_trips.head()

C:\Users\mihai\AppData\Local\Temp\ipykernel_14472\2902331393.py:1: UserWarning: Parsing dates in %d/%m/%Y %H:%M format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  short_trips = pd.read_csv("../data/processed/sn_shorttrips.csv", parse_dates=["ReservationTime", "StartTime", "EndTime"])


,BilID,PersonID,LatitudeStart,LongitudeStart,LatitudeEnd,LongitudeEnd,ReservationTime,StartTime,EndTime,Age,...,KmDriven,FromZoneID,ToZoneID,TripDurationHours,TripDurationMinutes,AvgSpeed,Month,Weekday,StartHour,HaversineDistance
0,WBY1Z21040V307905,32614,55.670268,12.586064,55.652954,12.544699,2017-08-16 14:40:00,2017-08-16 14:34:00,2017-08-16 14:52:00,32,...,5,103142,102851,0.300000,18.0,16.666667,August,Wednesday,14,3.231816
1,WBY1Z21040V308214,44363,55.687024,12.585952,55.726655,12.553914,2017-08-16 14:42:00,2017-08-16 14:38:00,2017-08-16 15:05:00,25,...,7,102212,157231,0.450000,27.0,15.555556,August,Wednesday,14,4.843874
2,WBY1Z21040V307869,37425,55.630274,12.648864,55.706866,12.587122,2017-07-31 20:31:00,2017-07-31 20:31:00,2017-08-01 09:13:00,30,...,18,185203,102341,12.700000,762.0,1.417323,July,Monday,20,9.358422
3,WBY1Z21090V308015,18497,55.700745,12.473436,55.630263,12.648955,2017-07-31 21:11:00,2017-07-31 21:04:00,2017-08-01 04:28:00,42,...,28,102651,185203,7.400000,444.0,3.783784,July,Monday,21,13.517087
4,WBY1Z21060V308019,28029,55.712655,12.561513,55.658484,12.536456,2017-07-31 21:57:00,2017-07-31 21:56:00,2017-08-01 00:04:00,26,...,7,102412,102841,2.133333,128.0,3.281250,July,Monday,21,6.226910


In [5]:
# Assuming df is your dataframe and it contains 'Weekday' and 'StartDate' columns
# You would use the function as follows:
short_trips.loc[:, 'StartTimeUpdated'] = update_start_date_vectorized(short_trips['Weekday'], short_trips['StartTime'])

In [6]:
short_trips.head()

,BilID,PersonID,LatitudeStart,LongitudeStart,LatitudeEnd,LongitudeEnd,ReservationTime,StartTime,EndTime,Age,...,FromZoneID,ToZoneID,TripDurationHours,TripDurationMinutes,AvgSpeed,Month,Weekday,StartHour,HaversineDistance,StartTimeUpdated
0,WBY1Z21040V307905,32614,55.670268,12.586064,55.652954,12.544699,2017-08-16 14:40:00,2017-08-16 14:34:00,2017-08-16 14:52:00,32,...,103142,102851,0.300000,18.0,16.666667,August,Wednesday,14,3.231816,2023-11-15 14:34:00
1,WBY1Z21040V308214,44363,55.687024,12.585952,55.726655,12.553914,2017-08-16 14:42:00,2017-08-16 14:38:00,2017-08-16 15:05:00,25,...,102212,157231,0.450000,27.0,15.555556,August,Wednesday,14,4.843874,2023-11-15 14:38:00
2,WBY1Z21040V307869,37425,55.630274,12.648864,55.706866,12.587122,2017-07-31 20:31:00,2017-07-31 20:31:00,2017-08-01 09:13:00,30,...,185203,102341,12.700000,762.0,1.417323,July,Monday,20,9.358422,2023-11-20 20:31:00
3,WBY1Z21090V308015,18497,55.700745,12.473436,55.630263,12.648955,2017-07-31 21:11:00,2017-07-31 21:04:00,2017-08-01 04:28:00,42,...,102651,185203,7.400000,444.0,3.783784,July,Monday,21,13.517087,2023-11-20 21:04:00
4,WBY1Z21060V308019,28029,55.712655,12.561513,55.658484,12.536456,2017-07-31 21:57:00,2017-07-31 21:56:00,2017-08-01 00:04:00,26,...,102412,102841,2.133333,128.0,3.281250,July,Monday,21,6.226910,2023-11-20 21:56:00


In [3]:
short_trips.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 310055 entries, 0 to 310054
Data columns (total 23 columns):
 #   Column               Non-Null Count   Dtype         
---  ------               --------------   -----         
 0   BilID                310055 non-null  object        
 1   PersonID             310055 non-null  int64         
 2   LatitudeStart        310055 non-null  float64       
 3   LongitudeStart       310055 non-null  float64       
 4   LatitudeEnd          310055 non-null  float64       
 5   LongitudeEnd         310055 non-null  float64       
 6   ReservationTime      310055 non-null  datetime64[ns]
 7   StartTime            310055 non-null  datetime64[ns]
 8   EndTime              310055 non-null  datetime64[ns]
 9   Age                  310055 non-null  int64         
 10  Gender               310055 non-null  object        
 11  BatteryStart         310055 non-null  float64       
 12  BatteryEnd           310055 non-null  float64       
 13  KmDriven      

In [4]:
main_concurrent_sn(short_trips, num_itineraries=3)